In [47]:
import requests
import pandas as pd
import json
from pprint import pprint
import time

In [42]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [3]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers


In [4]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"
locations = pd.read_csv(locations_csv)
#locations

In [5]:
locations["Latitude"] = locations["Latitude"].round(decimals=3)
locations["Longitude"] = locations["Longitude"].round(decimals=3)
#locations

In [6]:
#merge schedule with locations
merge = pd.merge(left=schedule, right=locations, how="left", left_on="City", right_on="Franchise")
merge.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Franchise,Latitude,Longitude
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,New York,40.43,-74.01
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,Philadelphia,39.57,-75.07
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,Miami,25.46,-80.12
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,New England,42.21,-71.04
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,Pittsburgh,40.26,-80.00


In [9]:
clean_df = merge[["Full Date","Time","time_conv","time_index","Week","City","Home Team","Winner","Latitude","Longitude"]]
clean_df["Starting Game Temp (F)"] = ""
clean_df["weatherDesc"] = ""
clean_df["cloudcover"] = ""
clean_df.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,42.21,-71.04,,,
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,40.26,-80.00,,,


In [10]:
# set up trial df
trial_df = clean_df.iloc[:3,:]
trial_df

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,


In [13]:
for index, row in trial_df.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        location = f"{lat},{lng}"
        time = row["time_index"]
        print(location)
        print(date)
        print(time)
    except IndexError:
        print("error")

40.43,-74.01
2008-09-04
18
39.57,-75.07
2008-09-07
12
25.46,-80.12
2008-09-07
12


In [54]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q="
for index, row in trial_df.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        #print(weather_test)
        weather_info = requests.get(weather_test).json()
        #pprint(weather_info)
        temp = weather_info['data']['weather']['hourly']['tempF']
        print(temp)
        #trial_df.loc[index, "Starting Game Temp (F)"] = weather_info["tempF"][time]
    except IndexError:
        print("error")
            

TypeError: list indices must be integers or slices, not str

In [15]:
trial_df.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,


In [ ]:
#http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,-87.38&date=2008-09-08&tp=1&format=json